## Installation

In [1]:
#pip install openpyxl

## Import 

In [ ]:
import cv2
import os.path
import sys
from pathlib import Path
import time 
import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image
import math
from openpyxl import Workbook
from openpyxl import load_workbook
import pandas as pd 
#import uuid

## Use right data sets  

In [ ]:
labels = ['Lundi']
full_List_coordonate = []

In [ ]:
#If the project does not contain an image file 
if not os.path.exists('images'):
    os.makedirs('images')
    for label in labels:
        os.makedirs('images\{}'.format(label))
elif not os.path.exists('DataSet.xlsx'):
    wb = Workbook()
    ws = wb.active
    
    ws['A1'] = 'NameMouv'
    ws['B1'] = 'SerieID'
    ws['C1'] = 'X'
    ws['D1'] = 'Y'
    ws['E1'] = 'Vector_X'
    ws['F1'] = 'Vector_Y'
    
    wb.save(filename = 'DataSet.xlsx')
    
else:
    pass

In [ ]:
def convertArrayToImage(array):
    w, h = 512, 512
    data = np.zeros((h, w, 3), dtype=np.uint8)
    data[0:256, 0:256] = [255, 0, 0] # red patch in upper left
    img = Image.fromarray(array, 'RGB')
    img.save('my.png')
    img.show()

In [ ]:
#Hand 
class Hand(object):
    def __init__(self, frame, contouring):
        self.frame = frame 
        self.contouring = contouring
        self.contours = []
        self.outline = self.draw_outline()  
        
    
    def draw_outline(self, min_area=10000, color=(0, 255, 0), thickness=2):
        #find contour 
        contours,_  = cv2.findContours(self.contouring, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        palm_area = 0
        flag = None
        cnt = None
        for (i, c) in enumerate(contours):
            area = cv2.contourArea(c)
            if area > palm_area:
                palm_area = area
                flag = i
        if flag is not None and palm_area > min_area:
            cnt = contours[flag]
            self.contours = cnt
            cpy = self.frame.copy()
            cv2.drawContours(cpy, [cnt], 0, color, thickness)
            return cpy
        else:
            return self.frame
    
    def get_center_of_mass(self):
        if len(self.contours) == 0:
            return None
        M = cv2.moments(self.contours)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        return (cX, cY)
    
    

In [ ]:
def focusBox(source = 0):
    cap = cv2.VideoCapture(source)
    color = (255, 0, 0)
    while(True):
        _,frame = cap.read()
        frame = cv2.flip(frame, 1)
        frame = cv2.resize(frame, (1000, 600))
        cv2.rectangle(frame, (150,150), (220, 220), color, 2)
        cv2.imshow('focusBox', frame)
        box = frame[153:218, 153:218]
        k = cv2.waitKey(1)
        if k%256 == 32:
            #obj_color = cv2.imread('C:\\Users\\jujub\\CNN\\unnamed.jpg')
            obj_color = box
            break 
    obj_color_hsv = cv2.cvtColor(obj_color, cv2.COLOR_BGR2HSV)
    # calculating object histogram
    obj_hist = cv2.calcHist([obj_color_hsv], [0, 1], None,
                               [180, 256], [0, 180, 0, 256])
    #Normalize histogram
    cv2.normalize(obj_hist, obj_hist, 0, 255, cv2.NORM_MINMAX)     
    cv2.destroyAllWindows()
    cap.release()
    return obj_hist

def track_object(frame, obj_hist):
    frame_color = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    dst = cv2.calcBackProject([frame_color],[0,1],obj_hist,[0,180,0,256],1)
    # Now convolute with circular disc
    disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(9, 9))
    cv2.filter2D(dst,-1,disc,dst)
    ret, tresh = cv2.threshold(dst, 0, 256, cv2.THRESH_BINARY)
    kernel = None
    eroded = cv2.erode(tresh, kernel, iterations=2)
    dilated = cv2.dilate(eroded, kernel, iterations=2)
    closing = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, kernel)
    return closing
    
def detect_hand(frame, hist):
    contouring = track_object(frame, hist)
    return Hand(frame, contouring)

def detect_face(frame, block=False, colour=(0, 0, 0)):
    fill = [1, -1][block]
    face_cascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)
    area = 0
    X = Y = W = H = 0
    for (x, y, w, h) in faces:
        if w * h > area:
            area = w * h
            X, Y, W, H = x, y, w, h
    cv2.rectangle(frame, (X, Y), (X + W, Y + H), colour, fill)

In [ ]:
#saved the data in an excel file
def record_coordonate(temp_list_coordonate, label, numvid, full_List_coordonate):
    count = 0
    Name_Mouv = label+'_'+ str(numvid)
    #dict_coordonate[label] = {} if dict_coordonate.get(label) is None else dict_coordonate
    #dict_coordonate if key_second in dict_coordonate[label] else dict_coordonate[label][key_second] = []
#     if key_second not in dict_coordonate[label]:
#         dict_coordonate[label][key_second] = []
    for cor in temp_list_coordonate:
        if not isinstance(cor, type(None)):
            list_cor = list(cor)
            list_cor.append(count)
            list_cor.append(Name_Mouv)
            cor = tuple(list_cor)
            full_List_coordonate.append(cor)
            count += 1 
    return full_List_coordonate

In [ ]:
def videoCapture(labels,  full_List_coordonate, createData = False):
    hist = focusBox()
    if createData == True:
        temp_list_coordonate = []
        cap = cv2.VideoCapture(0)
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        capture_duration = 4 + 3
        num_video = 2
        for label in labels: 
            print('collecting for {}'.format(label))
            save_path = "path".format(label)
            #print(save_path)
            for numvid in range(num_video):
                start_time = time.time()
                file_name = label + '_' + str(numvid)+'.mp4'
                save_path = save_path + file_name
                out = cv2.VideoWriter(save_path,fourcc, 20.0, (640,480))
                count = 0 
                while(int(time.time() - start_time) < capture_duration ):
                    re,frames = cap.read()
                    hand = detect_hand(frames, hist)
                    if re == True:
                        frames = detect_face(frames, block=True)
                        frames = hand.draw_outline()
                        com = hand.get_center_of_mass()
                        if com:
                            frames = cv2.circle(hand.outline , com, 10, (255, 0, 0), -1)
                        temp_list_coordonate.append(com)
                        cv2.imshow('frame', frames)
                        out.write(frames)
                        count += 1 
                        k = cv2.waitKey(5)
                        # Press 'q' to exit
                        if k == ord('q'):
                            break 
                    else:
                        break
                if save_path:
                    save_path = save_path.rpartition('\\')[0]+save_path.rpartition('\\')[1]
                full_List_coordonate = record_coordonate(temp_list_coordonate, label, numvid, full_List_coordonate)
                temp_list_coordonate.clear()
                #print(full_List_coordonate)
                #print(temp_list_coordonate)
        # When everything done, release the capture
        cap.release()
        out.release()
        cv2.destroyAllWindows()
    else:
        pass 
    return full_List_coordonate

In [ ]:
full_List_coordonate = videoCapture(labels, full_List_coordonate, createData = True)

In [ ]:
def saveDataSet(full_List_coordonate):
    wb = load_workbook('DataSet.xlsx')
    ws = wb['Sheet']
    row = ws.max_row + 1
    col = 1

    for x , y, serieid, namemouv in full_List_coordonate:
        ws.cell(row= row, column= col).value = namemouv
        ws.cell(row= row, column= col+1).value = serieid
        ws.cell(row= row, column= col+2).value = x
        ws.cell(row= row, column= col+3).value = y
        row += 1 
    
    wb.save('DataSet.xlsx')

def addVector(df):
    book = load_workbook('DataSet.xlsx')
    ws = book['Sheet']
    dfVector = df[["Vector_X", "Vector_Y"]]
    writer = pd.ExcelWriter('DataSet.xlsx', engine = 'openpyxl')
    writer.book  = book
    max_row = max((c.row for c in ws['C'] if c.value is not None))
    dfVector.to_excel(writer, sheet_name = 'Sheet', startrow = max_row, 
                      columns = ["Vector_X", "Vector_Y"], index = False,header= False)
    writer.save()

In [ ]:
def createVector(df):
    #listMouv = list(dict.fromkeys(df['NameMouv']))
    grouped = df.groupby('NameMouv')
    diff_vectorx = lambda x: x['X'].diff()
    diff_vectory = lambda x: x['Y'].diff()
    df['Vector_X'] = grouped.apply(diff_vectorx).reset_index(0, drop=True)
    df['Vector_Y'] = grouped.apply(diff_vectory).reset_index(0, drop=True)
    return df

In [ ]:
saveDataSet(full_List_coordonate)
dt = pd.read_excel('DataSet.xlsx',dtype={'NameMouv' : str, 
                  'SerieID' : int ,'X': int, 'Y': int })
df = pd.DataFrame(dt)
df = createVector(df)

In [ ]:
addVector(df)

In [ ]:
# for i in dict_coordonate.keys():
#     print(i)
    
# print(dict_coordonate[i])

# df_coordonate = storeInDataframe(dict_coordonate)